# RLCT estimation

In [1]:
from devinterp.slt.sampler import Sampler
from devinterp.evals import SamplerEvaluator
from devinterp.ops.storage import CheckpointerConfig

from icl.config import get_config
from icl.evals import ICLEvaluator
from icl.utils import set_seed

In [4]:
config = get_config(task_config={"num_tasks": 64})

import yaml
print(yaml.dump(config.model_dump()))

{'task_size': 8, 'max_examples': 16, 'num_tasks': 64, 'noise_variance': 0.25, 'embed_size': 128, 'mlp_size': 128, 'num_heads': 2, 'num_layers': 8}
batch_size: 256
checkpointer_config:
  bucket_name: devinterp
  device: cpu
  local_root: null
  project_dir: icl/ntasks-64-task-dbb989-opt-f7c569-sched-c4766a
criterion: cross_entropy
device: cuda
eval_batch_size: 2048
logger_config:
  entity: null
  metrics: null
  out_file: null
  project: null
  run_id: null
  stdout: false
  use_df: false
num_steps: 500000
num_training_samples: 128000000
optimizer_config:
  betas: !!python/tuple
  - 0.9
  - 0.999
  elasticity: null
  lr: 0.001
  momentum: null
  noise_level: null
  num_samples: null
  optimizer_type: Adam
  temperature: null
  weight_decay: 0.0
scheduler_config:
  T_max: null
  anneal_strategy: linear
  cycle_momentum: false
  div_factor: 249999.0
  eta_min: null
  final_div_factor: 249999.0
  gamma: null
  last_epoch: -1
  lr_lambda: null
  max_lr: 0.001
  milestones: null
  pct_start:

In [5]:
# Get the checkpoints
checkpointer = config.checkpointer_config.factory().sync()
checkpointer.file_ids = sorted([int(x) for x in checkpointer.file_ids])
checkpointer.file_ids

/home/paperspace/Projects/devinterp/devinterp/ops/storage.py:91: UserWarning: Using default key_to_id. This yields a string, which may not be what you want.
  warnings.warn(


[0,
 1,
 2,
 3,
 4,
 6,
 8,
 11,
 14,
 19,
 24,
 32,
 42,
 55,
 72,
 94,
 124,
 162,
 211,
 276,
 362,
 473,
 618,
 808,
 1056,
 1381,
 1805,
 2359,
 3084,
 4031,
 5269,
 6887,
 9003,
 10204,
 11767,
 15381,
 20104,
 20408,
 26279,
 30612,
 34349,
 40816,
 44897,
 51020,
 58684,
 61224,
 71428,
 76706,
 81632,
 91836,
 100262,
 102040,
 112244,
 122448,
 131051,
 132653,
 142857,
 153061,
 163265,
 171296,
 173469,
 183673]

In [6]:
set_seed(0)

# initialise model
model = config.task_config.model_factory().to(config.device)

# initialise 'pretraining' data source (for training on fixed task set)
pretrain_dist = config.task_config.pretrain_dist_factory().to(config.device)

# initialise 'true' data source (for evaluation, including unseen tasks)
true_dist = config.task_config.true_dist_factory().to(config.device)

# initialise evaluations
evaluator = ICLEvaluator(
    pretrain_dist=pretrain_dist,
    true_dist=true_dist,
    max_examples=config.task_config.max_examples,
    eval_batch_size=config.eval_batch_size,
)

# Load model
model.load_state_dict(checkpointer[-1]["model"])

evaluator(model)

{'pretrain/mse': 22.708744049072266,
 'pretrain/delta_dmmse': tensor(21.2075, device='cuda:0'),
 'pretrain/delta_ridge': tensor(18.7974, device='cuda:0'),
 'pretrain/token/0': 22.16046905517578,
 'pretrain/token/1': 22.93655776977539,
 'pretrain/token/2': 22.1809024810791,
 'pretrain/token/3': 23.153139114379883,
 'pretrain/token/4': 23.117576599121094,
 'pretrain/token/5': 23.660362243652344,
 'pretrain/token/6': 22.545001983642578,
 'pretrain/token/7': 21.922630310058594,
 'pretrain/token/8': 22.393159866333008,
 'pretrain/token/9': 22.808799743652344,
 'pretrain/token/10': 22.767139434814453,
 'pretrain/token/11': 23.420642852783203,
 'pretrain/token/12': 23.997772216796875,
 'pretrain/token/13': 22.09250259399414,
 'pretrain/token/14': 21.238628387451172,
 'pretrain/token/15': 22.944610595703125,
 'true/mse': 20.287052154541016,
 'true/delta_dmmse': tensor(19.2558, device='cuda:0'),
 'true/delta_ridge': tensor(17.3636, device='cuda:0'),
 'true/token/0': 20.61550521850586,
 'true/to

In [4]:
set_seed(0)

# initialise model
model_1 = config.task_config.model_factory().to(config.device)

# initialise 'pretraining' data source (for training on fixed task set)
pretrain_dist_1 = config.task_config.pretrain_dist_factory().to(config.device)

# initialise 'true' data source (for evaluation, including unseen tasks)
true_dist_1 = config.task_config.true_dist_factory().to(config.device)

In [5]:
set_seed(0)

# initialise model
model_2 = config.task_config.model_factory().to(config.device)

# initialise 'pretraining' data source (for training on fixed task set)
pretrain_dist_2 = config.task_config.pretrain_dist_factory().to(config.device)

# initialise 'true' data source (for evaluation, including unseen tasks)
true_dist_2 = config.task_config.true_dist_factory().to(config.device)

True

In [22]:
import json
import random
import warnings

import numpy as np
import torch

json.dumps({
    "task_size": 8,
    "max_examples": 16,
    "num_tasks": 1,
    "noise_variance": 0.25,
    "embed_size": 128,
    "mlp_size": 128,
    "num_heads": 2,
    "num_layers": 8,
}, sort_keys=True)


'{"embed_size": 128, "max_examples": 16, "mlp_size": 128, "noise_variance": 0.25, "num_heads": 2, "num_layers": 8, "num_tasks": 1, "task_size": 8}'